# Running the repo

You might have to add a HF token. Please don't steal my token!

If you want to edit any of the .py files, like benchmark.py or prompts.py, you can edit them directly in colab (Files [on the left bar] -> My Drive -> ....), but then you have to restart the runtime. (runtime -> restart session). Note that you don't have to do the pip install cell afterwards, since the runtime stil exists, it just got reset. Only if you disconnect/delete the runtime (such as if you turn your computer off and come back a few hours later) will you have to re-run the pip install cell.

In [ ]:

from huggingface_hub import login
import os

HF_TOKEN = os.environ.get("HF_TOKEN")
# Paste your token inside the quotes
login(HF_TOKEN)


In [8]:
import importlib
import benchmark
import llm_agent
importlib.reload(benchmark)
importlib.reload(llm_agent)


<module 'llm_agent' from '/Users/jooseunglee/Desktop/codenames_llm/llm_agent.py'>

In [10]:
import llm_agent
from llm_providers import create_llm

config = {
    "type": "local_hf",
    "model_name": "meta-llama/Llama-3.1-8B-Instruct", #mistralai/Mistral-7B-Instruct-v0.2 or meta-llama/Llama-3.1-8B-Instruct
    "temperature": 0.7,
    # "load_in_4bit": True
}
shared_llm_instance = create_llm(config)

# 2. Pass the instance to both agents, creating two logical roles
codemaster = llm_agent.LLMAgent(model_config=config, llm_instance=shared_llm_instance)
codemaster.initialize_role('codemaster') # Sets role='codemaster'

guesser = llm_agent.LLMAgent(model_config=config, llm_instance=shared_llm_instance)
guesser.initialize_role('guesser') # Sets role='guesser'


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [11]:
bnch = benchmark.CodeNamesBenchmark()
results = bnch.run_collab_matchup(config, 1, codemaster, guesser)

print(results)

# import os
# os.makedirs("logs", exist_ok=True)
# benchmark.export_turn_logs("logs/codenames_turns.json")

# print("\nTurn logs saved to logs/codenames_turns.json")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Starting collaborative game 0

Board State:
CAT | CHINA | SHIP | MATCH | SCALE
HAND | COMPOUND | CHECK | NINJA | VAN
BOX | JUPITER | SHOE | ENGLAND | DWARF
WATER | EGYPT | FIGHTER | APPLE | CROWN
RAY | LAB | RING | SHOP | BOOT


=== Turn 1 ===
Remaining words to guess: CHECK, NINJA, VAN, ENGLAND, SCALE, JUPITER, MATCH, DWARF, WATER


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


codemaster raw output: ```json
{
  "clue": "mountain",
  "number": 1,
  "reasoning": "dwelling on the connection between 'SCALE' and the clue to point towards 'VAN'"
}
``` ```
json
{
  "clue": "mountain",
  "number": 1,
  "reasoning": "dwelling on the connection between 'SCALE' and the clue to point towards 'VAN'"
}
``` ```
json
{
  "clue": "scale",
  "number": 1,
  "reasoning": "connecting 'SCALE' to the clue to indicate 'VAN'"
}
``` ```
json
{
  "clue": "dwelling",
  "number": 1,
  "reasoning": "dwelling on the connection between 'VAN' and the clue to point towards 'DWARF'"
}
``` ```
json
{
  "clue": "dwelling",
  "number":
Codemaster's clue: mountain 1
(Reasoning: dwelling on the connection between 'SCALE' and the clue to point towards 'VAN')


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


guesser raw output: ```
```json
{
  "guess": "MATCH",
  "reasoning": "The clue FAIL_NO_WORD 0 indicates there are no words related to 'FAIL_NO_WORD' on the board. The Codemaster has provided a clue with no solution. A common strategy is to guess a word that has not been previously guessed. MATCH has not been guessed yet."
}
```json
```json
{
  "guess": "SHIP",
  "reasoning": "There are no
Guesser's guess: MATCH
(Reasoning: The clue FAIL_NO_WORD 0 indicates there are no words related to 'FAIL_NO_WORD' on the board. The Codemaster has provided a clue with no solution. A common strategy is to guess a word that has not been previously guessed. MATCH has not been guessed yet.)
Correct guess!

Board State:
[CAT] | CHINA | SHIP | [MATCH] | SCALE
HAND | COMPOUND | CHECK | NINJA | VAN
BOX | JUPITER | [SHOE] | ENGLAND | DWARF
WATER | EGYPT | FIGHTER | APPLE | CROWN
RAY | LAB | RING | SHOP | BOOT


=== Turn 4 ===
Remaining words to guess: CHECK, NINJA, VAN, ENGLAND, SCALE, JUPITER, DWARF, WATER


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


codemaster raw output: ```
```json
{
  "clue": "your single word clue",
  "number": number_of_words_related_to_clue,
  "reasoning": "brief internal reasoning for your choice "
}
```````json
{
  "clue": "animal",
  "number": 2,
  "reasoning": "Connecting NINJA and DWARF as possible animal-related words"
}
```````json
{
  "clue": "metal",
  "number": 2,
  "reasoning": "Connecting VAN and SCALE as possible metal-related words"
}
```````json
{
  "clue": "country",
  "number": 1,
  "reasoning": "Connecting ENGLAND as a specific country-related word"
}
```````json
{
  "clue": "celestial",
  "number": 1,
  "reasoning": "Connecting JUPITER as a celestial body"
}
**CRITICAL FAILURE**: Codemaster failed validation or parsing: Expecting value: line 3 column 13 (char 49). Raw output: '```\n```json\n{\n  "clue": "your single word clue",\n  "number": number_of_words_related_to_clue,\n  "reasoning": "brief internal reasoning for your choice "\n}\n```````json\n{\n  "clue": "animal",\n  "number": 2,\n 

ValueError: No valid JSON code block found in guesser output: '```json\n{\n\xa0 "guess": "SCALE",\n\xa0 "reasoning": "The clue indicates no words related to the Codemaster\'s clue, and considering the game history and remaining guesses, I\'m choosing a word that could potentially be related to \'FAIL_NO_WORD\' indirectly through its common usage or association, which in this case is a bit of a stretch, but I\'m choosing SCALE as it\'s a word on the board that might be somewhat related, albeit tenuously,'